In [1]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble,
)

from pyciemss.PetriNetODE.interfaces import load_and_sample_petri_model

In [2]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_2 = "test/models/AMR_examples/SIDARTHE.amr.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

In [3]:
def solution_mapping1(model1_solution: dict) -> dict:
    return model1_solution


def solution_mapping2(model2_solution: dict) -> dict:
    mapped_solution = {}
    mapped_solution["Susceptible"] = (
        model2_solution["Susceptible"]
        + model2_solution["Recognized"]
        + model2_solution["Threatened"]
    )

    mapped_solution["Infected"] = (
        model2_solution["Infected"]
        + model2_solution["Ailing"]
        + model2_solution["Diagnosed"]
    )

    # Model 1 doesn't include dead people, and implicitly assumes that everyone who is infected will recover.
    mapped_solution["Recovered"] = (
        model2_solution["Healed"] + model2_solution["Extinct"]
    )

    return mapped_solution


solution_mappings = [solution_mapping1, solution_mapping2]

## load_and_sample_petri_model

In [4]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints
)
# Save results
# samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [5]:
ensemble_samples

{'model_0/beta': tensor([0.0250, 0.0255, 0.0282, 0.0251, 0.0247, 0.0292, 0.0263, 0.0261, 0.0271,
         0.0292, 0.0274, 0.0259, 0.0268, 0.0255, 0.0293, 0.0295, 0.0264, 0.0252,
         0.0273, 0.0287, 0.0245, 0.0245, 0.0291, 0.0268, 0.0265, 0.0280, 0.0284,
         0.0291, 0.0290, 0.0293, 0.0246, 0.0286, 0.0282, 0.0268, 0.0282, 0.0269,
         0.0281, 0.0289, 0.0261, 0.0258, 0.0294, 0.0289, 0.0271, 0.0282, 0.0257,
         0.0293, 0.0286, 0.0286, 0.0260, 0.0265, 0.0282, 0.0286, 0.0282, 0.0256,
         0.0253, 0.0267, 0.0243, 0.0259, 0.0290, 0.0277, 0.0260, 0.0272, 0.0260,
         0.0262, 0.0295, 0.0290, 0.0280, 0.0268, 0.0279, 0.0274, 0.0270, 0.0283,
         0.0252, 0.0294, 0.0271, 0.0296, 0.0276, 0.0248, 0.0278, 0.0255, 0.0269,
         0.0269, 0.0269, 0.0285, 0.0269, 0.0270, 0.0276, 0.0258, 0.0277, 0.0292,
         0.0263, 0.0265, 0.0290, 0.0280, 0.0252, 0.0252, 0.0278, 0.0286, 0.0245,
         0.0284]),
 'model_0/gamma': tensor([0.1518, 0.1443, 0.1470, 0.1463, 0.1433, 0.1426, 

## load_and_calibrate_and_sample_petri_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
